In [1]:
## Comment out in Kaggle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install faiss-cpu

In [2]:
import json
import pandas as pd
import numpy as np
import faiss

ModuleNotFoundError: No module named 'faiss'

# Load data

In [ ]:
cols = ['id', 'authors', 'doi', 'title', 'abstract']
data = []
file_name = file_name = '/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json'
#file_name = '/content/drive/MyDrive/VT/Information Storage Project Files/arxiv-metadata-oai-snapshot.json'


with open(file_name, encoding='latin-1') as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['id'], doc['authors'], doc['doi'], doc['title'], doc['abstract']]
        data.append(lst)

df_data = pd.DataFrame(data=data, columns=cols)

print(df_data.shape)

df_data.head()

(2436004, 5)


,id,authors,doi,title,abstract
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Ileana Streinu and Louis Theran,None,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,Hongjun Pan,None,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,David Callan,None,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...


In [ ]:
df_data['prepared_text'] = '{title}: ' + df_data['title'] + ' {abstract}: ' + df_data['abstract']
df_data.head()

,id,authors,doi,title,abstract,prepared_text
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,{title}: Calculation of prompt diphoton produc...
1,0704.0002,Ileana Streinu and Louis Theran,None,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",{title}: Sparsity-certifying Graph Decompositi...
2,0704.0003,Hongjun Pan,None,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,{title}: The evolution of the Earth-Moon syste...
3,0704.0004,David Callan,None,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,{title}: A determinant of Stirling cycle numbe...
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,{title}: From dyadic $\Lambda_{\alpha}$ to $\L...


In [ ]:
df_data['prepared_text'][0]

'{title}: Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies {abstract}:   A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the 

# Embed data

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("thenlper/gte-base")
model.max_seq_length = 512

import time
start = time.time()

embeds = model.encode(list(df_data['prepared_text'][:10000]), batch_size=256, show_progress_bar=True)

end = time.time()
print(end-start) #20mins

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

193.88475584983826


In [ ]:
2436004
1000000

In [ ]:
embeds.shape

(100000, 768)

# Use Faiss to store the embeddings

In [ ]:
start = time.time()

index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(embeds, np.arange(len(df_data['prepared_text'][:10000])))

path = '/content/drive/MyDrive/gte-base_faiss_10000.index'

end = time.time()
print(end-start)

# Save index
faiss.write_index(index, path)

0.02446913719177246


# Load Faiss indices

In [ ]:

path = '/content/drive/MyDrive/gte-base_faiss_10000.index'
index = faiss.read_index(path)

# Get top k results

In [ ]:
def get_top_results(query, k):
  input_type_query = "search_query"

  query_embed = model.encode(query)

  top_k = index.search(np.array([query_embed]).astype('float32'), k)

  query_results = pd.DataFrame(data={'id': df_data.iloc[top_k[1][0]]['id'],
                                     'texts': df_data.iloc[top_k[1][0]]['prepared_text'],
                                     'authors': df_data.iloc[top_k[1][0]]['authors'],
                                     'doi': df_data.iloc[top_k[1][0]]['doi'],
                                     'distance': top_k[0][0]})

  return query_results

In [ ]:
get_top_results("quantum computer", 10)

,id,texts,authors,doi,distance
8173,0705.4171,{title}: Grover search algorithm {abstract}: ...,Eva Borbely,None,0.912940
7362,0705.3360,{title}: The Road to Quantum Artificial Intell...,Kyriakos N. Sgarbas,None,0.903458
8195,0705.4193,{title}: Lecture notes on Optical Quantum Comp...,Pieter Kok,10.1007/978-3-642-02871-7_7,0.902644
8305,0705.4303,{title}: Database Manipulation on Quantum Comp...,Ahmed Younes,None,0.896462
2032,0704.2033,{title}: The Quantum Interference Computer: an...,A.Y. Shiekh,10.1007/s10773-008-9664-7,0.889712
7335,0705.3333,{title}: Simulation of Quantum Algorithms with...,Peter Nyman,None,0.886327
8079,0705.4077,{title}: The power of quantum systems on a lin...,"Dorit Aharonov, Daniel Gottesman, Sandy Irani,...",10.1007/s00220-008-0710-3,0.885695
5175,0705.1173,{title}: Effective Physical Processes and Acti...,Ignazio Licata,None,0.881712
8019,0705.4017,{title}: Quantum pathology of static internal ...,Murat Cetinbas and Joshua Wilkie,10.1016/j.physleta.2007.05.074,0.880354
4633,0705.0631,{title}: Hybrid Quantum Cloning Machine {abstr...,"Satyabrata Adhikari, A. K. Pati, Indranil Chak...",None,0.880137


# Evaluation

In [ ]:
eval_data = pd.read_csv('/content/drive/MyDrive/VT/Information Storage Project Files/evaluation_data_500.csv')

In [ ]:
eval_data.head()

,Unnamed: 0,id,authors,doi,title,abstract,question1,question2
0,0,704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,- Calculation of diphoton production at hadron...,- Comparison of diphoton production from Higgs...
1,1,704.0002,Ileana Streinu and Louis Theran,NaN,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...","- How to characterize $(k,\ell)$-sparse graphs?","- How does the $(k,\ell)$-pebble game with col..."
2,2,704.0003,Hongjun Pan,NaN,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,- How has the Earth-Moon system evolved accord...,- What is the significance of the Roche's limi...
3,3,704.0004,David Callan,NaN,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,- How can acyclic single-source automata be en...,- What is the significance of Stirling cycle n...
4,4,704.0005,Wael Abu-Shammala and Alberto Torchinsky,NaN,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,- How to compute the $\Lambda_{\alpha}$ norm u...,- How are the Hardy spaces $H^p(R^N)$ describe...


In [ ]:
eval_data['question1'][0]

'- Calculation of diphoton production at hadron colliders'

In [ ]:
get_top_results(eval_data['question1'][0], 10)

,id,texts,authors,doi,distance
0,0704.0001,{title}: Calculation of prompt diphoton produc...,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,0.793048
293,0704.0294,{title}: QED x QCD Resummation and Shower/ME M...,B.F.L. Ward and S.A. Yost,None,0.683034
105,0704.0106,{title}: Multiple Parton Scattering in Nuclei:...,"Andreas Schafer, Xin-Nian Wang and Ben-Wei Zhang",10.1016/j.nuclphysa.2007.06.009,0.671896
317,0704.0318,{title}: Effects of Dirac sea on pion propagat...,Subhrajyoti Biswas and Abhee K. Dutt-Mazumder,10.1103/PhysRevC.77.045201,0.671161
36,0704.0037,{title}: The discrete dipole approximation for...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...",10.1016/j.jqsrt.2007.01.033,0.667122
425,0704.0426,{title}: Feedback from first radiation sources...,"Leonid Chuzhoy, Michael Kuhlen, Paul R. Shapiro",10.1086/521438,0.662461
495,0704.0496,{title}: Fusion process studied with preequili...,"C\'edric Simenel (SPhN, GANIL), Philippe Choma...",10.1103/PhysRevC.76.024609,0.661005
32,0704.0033,{title}: Convergence of the discrete dipole ap...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...",10.1364/JOSAA.23.002578 10.1364/JOSAA.32.002407,0.660479
135,0704.0136,{title}: Compounding Fields and Their Quantum ...,Zihua Weng,None,0.658491
151,0704.0152,{title}: Kinetic equation for finite systems o...,"V.I. Abrosimov, D.M. Brink, A.Dellafiore, F. M...",10.1016/j.nuclphysa.2007.11.009,0.657107


## top k = 1 results

In [ ]:
k =  1
true_false = []
for i in range(len(eval_data)):
  question = eval_data['question1'][i]
  correct_answer = eval_data['id'][i]
  results = get_top_results(question, k)
  result_ids = [float(j) for j in results['id'].values]
  # print(correct_answer)
  # print(result_ids)
  true_false.append(correct_answer in result_ids)
  print(i)

for i in range(len(eval_data)):
  try:
    question = eval_data['question2'][i]
    correct_answer = eval_data['id'][i]
    results = get_top_results(question, k)
    result_ids = [float(j) for j in results['id'].values]
    # print(correct_answer)
    # print(result_ids)
    true_false.append(correct_answer in result_ids)
    print(i)
  except:
    print(i, " - error")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
all(true_false)

False

In [ ]:
sum(true_false) / len(true_false)

0.9914529914529915

In [ ]:
len(true_false)

936

## top k = 3 results

In [ ]:
k =  3
true_false = []
for i in range(len(eval_data)):
  question = eval_data['question1'][i]
  correct_answer = eval_data['id'][i]
  results = get_top_results(question, k)
  result_ids = [float(j) for j in results['id'].values]
  # print(correct_answer)
  # print(result_ids)
  true_false.append(correct_answer in result_ids)
  print(i)

for i in range(len(eval_data)):
  try:
    question = eval_data['question2'][i]
    correct_answer = eval_data['id'][i]
    results = get_top_results(question, k)
    result_ids = [float(j) for j in results['id'].values]
    # print(correct_answer)
    # print(result_ids)
    true_false.append(correct_answer in result_ids)
    print(i)
  except:
    print(i,  " - error")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
all(true_false)

False

In [ ]:
sum(true_false) / len(true_false)

0.9989316239316239

## top k = 5 results

In [ ]:
k =  5
true_false = []
for i in range(len(eval_data)):
  question = eval_data['question1'][i]
  correct_answer = eval_data['id'][i]
  results = get_top_results(question, k)
  result_ids = [float(j) for j in results['id'].values]
  # print(correct_answer)
  # print(result_ids)
  true_false.append(correct_answer in result_ids)
  print(i)

for i in range(len(eval_data)):
  try:
    question = eval_data['question2'][i]
    correct_answer = eval_data['id'][i]
    results = get_top_results(question, k)
    result_ids = [float(j) for j in results['id'].values]
    # print(correct_answer)
    # print(result_ids)
    true_false.append(correct_answer in result_ids)
    print(i)
  except:
    print(i,  " - error")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
all(true_false)

True

In [ ]:
sum(true_false) / len(true_false)

1.0